In [123]:
import pandas as pd
import numpy as np
import os
from cassandra.cluster import Cluster
from datetime import datetime
import plotly.graph_objects as go
import settrade_v2
from settrade_v2 import Investor
import time

In [124]:
cluster = Cluster(['127.0.0.1']) 
session = cluster.connect()

In [125]:
investor = Investor(
                app_id="rirpANWTJCOHVgnn",                                 
                app_secret="P7iCFYB6JX0cdLXogMwU0J2yNsrpne/nzX2pFy6kiA8=", 
                broker_id="SANDBOX",
                app_code="SANDBOX",
                is_auto_queue = False)

deri = investor.Derivatives(account_no="wanneeba-D")            

account_info = deri.get_account_info()

account_info       

{'creditLine': 2000000000.0,
 'excessEquity': 2000000000.0,
 'cashBalance': 2000000000.0,
 'equity': 2000000000.0,
 'totalMR': 0.0,
 'totalMM': 0.0,
 'totalFM': 0.0,
 'callForceFlag': 'No',
 'callForceMargin': 0.0,
 'liquidationValue': 2000000000.0,
 'depositWithdrawal': 0.0,
 'callForceMarginMM': 0.0,
 'initialMargin': 0.0,
 'closingMethod': 'Auto Net'}

In [126]:
#Get Candlestick
market = investor.MarketData()
res = market.get_candlestick(
symbol="AOT",
interval="1d",
normalized=True,)

In [127]:
if isinstance(res, dict) and "data" in res:
    raw_data = res["data"]
else:
    raw_data = res  # กรณี res เป็น list เลย
df = pd.DataFrame(raw_data)
df

,lastSequence,time,open,high,low,close,volume,value
0,349695,1677603600,73.00,73.50,72.50,72.50,13912341,1.013501e+09
1,349695,1677690000,72.50,72.50,71.25,71.75,34605326,2.487860e+09
2,349695,1677776400,71.50,71.50,70.25,70.50,25367513,1.791836e+09
3,349695,1678122000,70.75,71.00,69.75,69.75,26164171,1.837748e+09
4,349695,1678208400,69.25,70.00,69.00,69.75,32814418,2.278454e+09
...,...,...,...,...,...,...,...,...
495,349695,1742403600,65.25,70.00,65.00,70.00,4405101,2.908413e+08
496,349695,1742490000,69.00,91.25,59.25,67.00,4222400,2.981925e+08
497,349695,1742749200,64.75,65.25,64.00,65.25,2254101,1.456885e+08
498,349695,1742835600,52.00,52.00,50.00,52.00,314400,1.621875e+07


In [128]:
session.set_keyspace('data_stock')

In [117]:
session.execute("""
    CREATE TABLE IF NOT EXISTS candlestick_data (
        symbol text,
        time timestamp,
        open_price float,
        high_price float,
        low_price float,
        close_price float,
        volume bigint,
        value float,
        PRIMARY KEY (symbol, time)
    ) WITH CLUSTERING ORDER BY (time ASC);
""")
print("✅ Keyspace และ Table พร้อมใช้งาน!")

✅ Keyspace และ Table พร้อมใช้งาน!


In [129]:
# ✅ โหลดรายชื่อหุ้นทั้งหมด
try:
    symbols_df = pd.read_excel("get_stcok_name/stocth_names.xlsx")
    symbols = symbols_df['หลักทรัพย์'].dropna().tolist()
except FileNotFoundError:
    print("❌ ไม่พบไฟล์ 'stocth_names.xlsx' ใช้รายชื่อหุ้นที่กำหนดแทน")
    symbols = ["PTT", "AOT", "SCB", "CPALL", "ADVANC"]

# ✅ ฟังก์ชันดึง candlestick จาก API
def get_candlestick_data(symbol):
    try:
        res = market.get_candlestick(  # ✅ ต้องกำหนด market ก่อนใช้
            symbol=symbol,
            interval="1d",
            limit=1000,
            normalized=True
        )
        return res
    except Exception as e:
        print(f"❌ ไม่สามารถดึงข้อมูลของ {symbol}: {e}")
        return None

# ✅ ฟังก์ชัน insert ข้อมูลลง Cassandra
def insert_financial_data(symbol, res):
    if not res or 'time' not in res:
        print(f"⚠️ ไม่มีข้อมูลสำหรับ {symbol}")
        return

    for i in range(len(res["time"])):
        # ✅ แปลง timestamp ให้ถูกต้อง
        timestamp = datetime.fromtimestamp(res["time"][i])

        session.execute("""
            INSERT INTO candlestick_data (
                symbol, time, open_price, high_price, low_price,
                close_price, volume, value
            ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
        """, (
            symbol,
            timestamp,
            res["open"][i],
            res["high"][i],
            res["low"][i],
            res["close"][i],
            res["volume"][i],
            res["value"][i]
        ))

    print(f"✅ เพิ่มข้อมูล {len(res['time'])} รายการของหุ้น {symbol} สำเร็จ!")

# ✅ ดึงและบันทึกข้อมูลทุกหุ้น
for symbol in symbols:
    print(f"\n📦 กำลังโหลดข้อมูล {symbol} ...")
    res = get_candlestick_data(symbol)
    insert_financial_data(symbol, res)
    time.sleep(2)  # 🕒 ลดโอกาสถูก block จาก API

print("\n🎉 เสร็จสิ้นการโหลดข้อมูลทั้งหมด!")


📦 กำลังโหลดข้อมูล 24CS ...
✅ เพิ่มข้อมูล 587 รายการของหุ้น 24CS สำเร็จ!

📦 กำลังโหลดข้อมูล 2S ...
✅ เพิ่มข้อมูล 711 รายการของหุ้น 2S สำเร็จ!

📦 กำลังโหลดข้อมูล 3BBIF ...
✅ เพิ่มข้อมูล 713 รายการของหุ้น 3BBIF สำเร็จ!

📦 กำลังโหลดข้อมูล A ...
✅ เพิ่มข้อมูล 712 รายการของหุ้น A สำเร็จ!

📦 กำลังโหลดข้อมูล A5 ...
✅ เพิ่มข้อมูล 710 รายการของหุ้น A5 สำเร็จ!

📦 กำลังโหลดข้อมูล AAI ...
✅ เพิ่มข้อมูล 576 รายการของหุ้น AAI สำเร็จ!

📦 กำลังโหลดข้อมูล AAV ...
✅ เพิ่มข้อมูล 719 รายการของหุ้น AAV สำเร็จ!

📦 กำลังโหลดข้อมูล ABM ...
✅ เพิ่มข้อมูล 709 รายการของหุ้น ABM สำเร็จ!

📦 กำลังโหลดข้อมูล ACAP ...
✅ เพิ่มข้อมูล 238 รายการของหุ้น ACAP สำเร็จ!

📦 กำลังโหลดข้อมูล ACC ...
✅ เพิ่มข้อมูล 710 รายการของหุ้น ACC สำเร็จ!

📦 กำลังโหลดข้อมูล ACE ...
✅ เพิ่มข้อมูล 711 รายการของหุ้น ACE สำเร็จ!

📦 กำลังโหลดข้อมูล ACG ...
✅ เพิ่มข้อมูล 706 รายการของหุ้น ACG สำเร็จ!

📦 กำลังโหลดข้อมูล ADB ...
✅ เพิ่มข้อมูล 709 รายการของหุ้น ADB สำเร็จ!

📦 กำลังโหลดข้อมูล ADD ...
✅ เพิ่มข้อมูล 710 รายการของหุ้น ADD สำเร็จ!

📦 กำล

In [143]:
name_stock = "A5"
rows = session.execute(f"SELECT * FROM candlestick_data WHERE symbol = '{name_stock}'")


In [144]:
df = pd.DataFrame(rows, columns=[
    'symbol', 'time', 'open_price', 'high_price', 'low_price', 'close_price', 'volume', 'value'
])

# ✅ แปลงชนิดเวลา + เรียงลำดับ
df['time'] = pd.to_datetime(df['time'])
df = df.sort_values('time').reset_index(drop=True)
df.head()


,symbol,time,open_price,high_price,low_price,close_price,volume,value
0,A5,2022-03-29,1.97,2.10,1.97,2.06,5.789567e+07,28666337
1,A5,2022-03-30,1.99,2.02,1.96,1.99,6.399037e+07,32160278
2,A5,2022-03-31,1.98,2.06,1.98,2.00,4.048799e+07,20186147
3,A5,2022-04-01,2.12,2.22,2.02,2.06,6.379672e+08,315307817
4,A5,2022-04-04,2.74,2.74,2.18,2.20,1.922967e+09,762254699


In [146]:
import plotly.graph_objects as go

# ✅ สร้างกราฟ Candlestick
fig = go.Figure()

# 🔹 แท่งเทียนราคา
fig.add_trace(go.Candlestick(
    x=df['time'],
    open=df['open_price'],
    high=df['high_price'],
    low=df['low_price'],
    close=df['close_price'],
    name=name_stock
))

# 🔹 Volume bar (แถบปริมาณการซื้อขาย)
fig.add_trace(go.Bar(
    x=df['time'],
    y=df['volume'],
    marker_color='rgba(128,128,128,0.3)',
    yaxis='y2',
    name='Volume'
))

# ✅ ปรับ layout ให้เหมือน TradingView
fig.update_layout(
    title=f'📈 {name_stock} - Candlestick Chart with Volume',
    xaxis_rangeslider_visible=False,
    yaxis=dict(title='Price'),
    yaxis2=dict(
        title='Volume',
        overlaying='y',
        side='right',
        showgrid=False
    ),
    height=600,
    template='plotly_white'
)

fig.show()
